### CONCERNS
1. list of directions/positions are never emptied during an episode, which makes the simulation slower and slower after each timestep

### IDEAS
other possible features:
1. robot is fallen down or not
2. distance to border (& which border?)

time optimalization:
1. clear last item from history every 4 timesteps (we only use the current and previous state and the one before that)
2. interval of states to be interpreted: skip N frames before evaluation next state
3. Since the rewards are so sparse, maybe use Imitation learning instead of DQN --> we are "experts" since we know the tactic of the blue bot. we can use this to teach our bot how to defeat the other agent.


In [1]:
from VisualModule import AgentEnvironment
from DQN_Agent import NeurosmashAgent

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

import gym
import math
import random
from collections import namedtuple
from itertools import count
from PIL import Image
import os

from stopwatch import Stopwatch


Using TensorFlow backend.


In [2]:
model_output_dir = "output/model_output/"

if not os.path.exists(model_output_dir):
    os.makedirs(model_output_dir)

show_images = False
skip_frames = 10
state_size =  11 # agent_pos, enemy_pos, vec_agent, vec_enemy, rel_pos_enemy, done
action_size = 3
episode_count = 1000
batch_size = 32
size       = 768         # Please check the Updates section above for more details
timescale  = 10           # Please check the Updates section above for more details

In [3]:
environment = AgentEnvironment(size=size, timescale=timescale)
agent = NeurosmashAgent(state_size = state_size, action_size = action_size) # action size: move in x or y direction, or do nothing

In [4]:
def direction(agent_path, enemy_path):
    A_X = (agent_path[-1] - np.array(agent_path[-2]))[0]
    A_Y = -(agent_path[-1] - np.array(agent_path[-2]))[1]
    E_X = (enemy_path[-1] - np.array(enemy_path[-2]))[0]
    E_Y = -(enemy_path[-1] - np.array(enemy_path[-2]))[1]
    return [A_X,A_Y],[E_X,E_Y]

def do_action(action, total_steps, eval_pic):
    stopwatch = Stopwatch() 
    stopwatch.start()
    info, reward, agent_coord, enemy_coord, following_state = environment.actionLoop(action, 0, eval_pic)
    stopwatch.stop()
    #print(f"Total time for do action: {stopwatch.duration}")
    if len(environment.agent_path) < 2:
        distance = 500 # Initial distance, only for initialisation
        agent_direction = [1,0] # By definition of facing each other
        enemy_direction = [-1,0]
    else:
        distance = np.sqrt(np.square(np.array(list(np.array(agent_coord)- np.array(enemy_coord))).sum(axis=0)))
        # Extract all variables 
        agent_direction, enemy_direction  = direction(environment.agent_path, environment.enemy_path)

    rel_pos_enemy = np.array(enemy_coord) - np.array(agent_coord)
    return info, reward, np.array(agent_coord), np.array(enemy_coord), agent_direction, enemy_direction, distance, rel_pos_enemy, following_state


In [5]:
def init_environment(env, agent_here):
    info, reward, state = env.reset() 
    agent_trajectories = []
    enemy_trajectories = []
    
    #for i in range(3):
    action = agent_here.act(3) # get next action
    #pre_state_img = np.flip(np.array(state).reshape(3,256,256).transpose(1,2,0),0)
    #step_number_now = i+1
    info, reward, agent_pos, enemy_pos, agent_direction, enemy_direction, distance, relative_pos_enemy, next_state = do_action(action, 1, True)  

    #post_state_img = np.flip(np.array(next_state).reshape(3,256,256).transpose(1,2,0),0)

    #agent_pos, enemy_pos = env_feat.coord(pre_state_img, post_state_img)
    agent_trajectories.append(list(agent_pos))
    enemy_trajectories.append(list(enemy_pos))
        
    return info, reward, next_state, agent_trajectories, enemy_trajectories, agent_direction, relative_pos_enemy, enemy_direction

In [ ]:
for e in range(episode_count):
    status, reward, next_state, agent_trajectories, enemy_trajectories, agent_dir, relative_pos_enemy, enemy_dir = init_environment(environment, agent)
    small_state = [agent_trajectories[-1][0], agent_trajectories[-1][1], enemy_trajectories[-1][0], enemy_trajectories[-1][1], [agent_dir[0]], [agent_dir[1]], [relative_pos_enemy[0]], [relative_pos_enemy[1]], [enemy_dir[0]], [enemy_dir[1]], False]#"agent direction", "relative position enemy", "enemy direction" ]

    done = False
    total_reward = 0
    total_timesteps = 1
    distances = []
    evaluate_frame = False

    while done == False:
        if (total_timesteps % skip_frames == 0) or (total_timesteps % skip_frames == skip_frames-1):
            evaluate_frame = True
        else:
            evaluate_frame = False
            
        action = agent.act(small_state) #step(info, reward, state)
        #print(f"agent chooses action: {action}")
        stopwatch = Stopwatch() 
        stopwatch.start()
        status, reward, agent_pos, enemy_pos, agent_dir, enemy_dir, distance, enemy_pos_rel, next_state = do_action(action, total_timesteps, evaluate_frame)   
        stopwatch.stop()
        #print(f"Total time for one step: {stopwatch.duration}")
        
        total_reward += reward

        if status == 1:
            print(f"Game is finished, \n your final reward is: {total_reward}, duration was {total_timesteps} timesteps")
            done = True
        
        agent_trajectories.append(list(agent_pos))
        enemy_trajectories.append(list(enemy_pos))
        distances.append(distance)
        
        done_list = [done]
        next_small_state = [agent_trajectories[-1][0], agent_trajectories[-1][1], enemy_trajectories[1][0], enemy_trajectories[1][1], agent_dir[0], agent_dir[1], enemy_pos_rel[0], enemy_pos_rel[1], enemy_dir[0], enemy_dir[1], done]  
    
        next_small_state = np.reshape(next_small_state, [1, state_size]) # why?

        if (total_timesteps % skip_frames == 0):
            agent.remember(small_state, action, reward, next_small_state, list(done_list))
        small_state = next_small_state
        total_timesteps += 1

    if len(agent.memory) > batch_size:
        agent.train(batch_size)
        print("train")

    if e % 50 == 0:
        agent.save(model_output_dir + "weights_"+ '{:04d}'.format(e) + ".hdf5")
            
    

total time for current frame (no analysis): 0.09792751399800181
total time for current frame (no analysis): 0.1336906570068095
total time for current frame (no analysis): 0.09594514699710999
total time for current frame (no analysis): 0.10340651799924672
total time for current frame (no analysis): 0.10780893599439878
total time for current frame (no analysis): 0.1037350400001742
total time for current frame (no analysis): 0.09733452799991937
total time for current frame (no analysis): 0.09870199600118212
total time for current frame (with analysis): 0.21520639800291974
total time for current frame (with analysis): 0.22749879999901168
total time for current frame (no analysis): 0.09990703200310236
total time for current frame (no analysis): 0.10336959100095555
total time for current frame (no analysis): 0.11721064699668204
total time for current frame (no analysis): 0.11634894600138068
total time for current frame (no analysis): 0.12471287899825256
total time for current frame (no analy

total time for current frame (with analysis): 0.20926895300362958
total time for current frame (with analysis): 0.21322263999900315
total time for current frame (no analysis): 0.09658912300074007
total time for current frame (no analysis): 0.09584077400359092
total time for current frame (no analysis): 0.09808383999916259
total time for current frame (no analysis): 0.09382491199357901
total time for current frame (no analysis): 0.1043580430050497
total time for current frame (no analysis): 0.09594875799666625
total time for current frame (no analysis): 0.11715469299815595
total time for current frame (no analysis): 0.1126160389976576
total time for current frame (with analysis): 0.2077095319982618
total time for current frame (with analysis): 0.20598235599754844
total time for current frame (no analysis): 0.09611334900546353
total time for current frame (no analysis): 0.095811473998765
total time for current frame (no analysis): 0.0976504729987937
total time for current frame (no analy

total time for current frame (with analysis): 0.2228830200037919
total time for current frame (no analysis): 0.10069968800235074
total time for current frame (no analysis): 0.10470013599842787
total time for current frame (no analysis): 0.10087439700146206
total time for current frame (no analysis): 0.10207299600006081
total time for current frame (no analysis): 0.09947811200254364
total time for current frame (no analysis): 0.0986333589971764
total time for current frame (no analysis): 0.09946922100061784
total time for current frame (no analysis): 0.10054397200292442
total time for current frame (with analysis): 0.21384436399966944
total time for current frame (with analysis): 0.21360188799735624
total time for current frame (no analysis): 0.10068428200611379
total time for current frame (no analysis): 0.10189773199817864
total time for current frame (no analysis): 0.10292179600219242
total time for current frame (no analysis): 0.10078085100394674
total time for current frame (no ana

total time for current frame (no analysis): 0.12149354400025913
total time for current frame (with analysis): 0.24866553399624536
total time for current frame (with analysis): 0.22841149900341406
total time for current frame (no analysis): 0.10252388700610027
total time for current frame (no analysis): 0.10225697299756575
total time for current frame (no analysis): 0.10291724499984412
total time for current frame (no analysis): 0.1547280859958846
total time for current frame (no analysis): 0.1008351360069355
total time for current frame (no analysis): 0.10320630700152833
total time for current frame (no analysis): 0.10009235600591637
total time for current frame (no analysis): 0.10946814400085714
total time for current frame (with analysis): 0.22907864800072275
total time for current frame (with analysis): 0.21451341600186424
total time for current frame (no analysis): 0.1021777510031825
total time for current frame (no analysis): 0.10027139500016347
total time for current frame (no an

total time for current frame (no analysis): 0.10129093999421457
total time for current frame (no analysis): 0.0992875730007654
total time for current frame (no analysis): 0.10125450699706562
total time for current frame (with analysis): 0.21687302000646014
total time for current frame (with analysis): 0.2142999330026214
total time for current frame (no analysis): 0.1011106860023574
total time for current frame (no analysis): 0.10016269700281555
total time for current frame (no analysis): 0.10317574000509921
total time for current frame (no analysis): 0.10186521399737103
total time for current frame (no analysis): 0.09883437500684522
total time for current frame (no analysis): 0.10177729700080818
total time for current frame (no analysis): 0.09928064100677148
total time for current frame (no analysis): 0.10234671899524983
total time for current frame (with analysis): 0.21384358400246128
total time for current frame (with analysis): 0.21584544899815228
total time for current frame (no an

...

total time for current frame (with analysis): 0.2137345499941148
total time for current frame (no analysis): 0.09989820699411212
total time for current frame (no analysis): 0.10091036599624204
total time for current frame (no analysis): 0.09747447800327791
total time for current frame (no analysis): 0.1002278509986354
total time for current frame (no analysis): 0.09795520699844928
total time for current frame (no analysis): 0.09935965199838392
total time for current frame (no analysis): 0.09728172500035726
total time for current frame (no analysis): 0.10260931799712125
total time for current frame (with analysis): 0.21265960299933795
total time for current frame (with analysis): 0.21123882599931676
total time for current frame (no analysis): 0.09852561899606371
total time for current frame (no analysis): 0.09713789999659639
total time for current frame (no analysis): 0.09557551899342798
total time for current frame (no analysis): 0.12837655899784295
total time for current frame (no ana

total time for current frame (no analysis): 0.10208465900359442
total time for current frame (no analysis): 0.09759995399508625
total time for current frame (with analysis): 0.23824206300196238
total time for current frame (with analysis): 0.21191445899603423
total time for current frame (no analysis): 0.09678970600361936
total time for current frame (no analysis): 0.0978614790001302
total time for current frame (no analysis): 0.09956771299766842
total time for current frame (no analysis): 0.0977850539929932
total time for current frame (no analysis): 0.09730783000122756
total time for current frame (no analysis): 0.0976453519979259
total time for current frame (no analysis): 0.09790775999863399
total time for current frame (no analysis): 0.09890214700135402
total time for current frame (with analysis): 0.20914236499811523
total time for current frame (with analysis): 0.2119750730053056
total time for current frame (no analysis): 0.09739505600009579
total time for current frame (no ana

total time for current frame (no analysis): 0.11055213500367245
total time for current frame (no analysis): 0.11886867200519191
total time for current frame (no analysis): 0.0993945270020049
total time for current frame (no analysis): 0.0986967669959995
total time for current frame (with analysis): 0.20811658800084842
total time for current frame (with analysis): 0.20831958900089376
total time for current frame (no analysis): 0.0984017410009983
total time for current frame (no analysis): 0.09707608300232096
total time for current frame (no analysis): 0.09762489000422647
total time for current frame (no analysis): 0.09764345699659316
total time for current frame (no analysis): 0.09560053300083382
total time for current frame (no analysis): 0.11198760999832302
total time for current frame (no analysis): 0.0970379580030567
total time for current frame (no analysis): 0.09725432000414003
total time for current frame (with analysis): 0.20756371899915393
total time for current frame (with ana

total time for current frame (no analysis): 0.10678020900377305
total time for current frame (no analysis): 0.09781334299623268
total time for current frame (no analysis): 0.09850467400246998
total time for current frame (no analysis): 0.1327868039952591
total time for current frame (no analysis): 0.09930994599562837
total time for current frame (with analysis): 0.20793123300245497
total time for current frame (with analysis): 0.2109817060045316
total time for current frame (no analysis): 0.09772986699681496
total time for current frame (no analysis): 0.09982657499494962
total time for current frame (no analysis): 0.11640748500212794
total time for current frame (no analysis): 0.11542765799822519
total time for current frame (no analysis): 0.09790752900153166
total time for current frame (no analysis): 0.10487349100003485
total time for current frame (no analysis): 0.1245848910039058
total time for current frame (no analysis): 0.09833742000046186
total time for current frame (with anal

total time for current frame (no analysis): 0.10120616899803281
total time for current frame (no analysis): 0.10187071799737168
total time for current frame (no analysis): 0.09919937999802642
total time for current frame (no analysis): 0.09869626100407913
total time for current frame (no analysis): 0.09887770999921486
total time for current frame (no analysis): 0.09987242500210414
total time for current frame (no analysis): 0.09759201099950587
total time for current frame (with analysis): 0.21507669799757423
total time for current frame (with analysis): 0.22261512000113726
total time for current frame (no analysis): 0.09846204099449096
total time for current frame (no analysis): 0.0991359219988226
total time for current frame (no analysis): 0.0997622089998913
total time for current frame (no analysis): 0.09903707099874737
total time for current frame (no analysis): 0.09885995199874742
total time for current frame (no analysis): 0.10279267699661432
total time for current frame (no analy